<a href="https://colab.research.google.com/github/Gaussiandra/Fake-News-classicifation/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
global_path = '/content/drive/My Drive/ML/RuCode20/Fake News classicifation/'

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
train_csv = pd.read_csv(global_path+'data/train.csv', delimiter='\t', skip_blank_lines=True)
test_csv = pd.read_csv(global_path+'data/test.csv', delimiter='\t')

In [ ]:
import torch

class FNDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]) 
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
test_texts = test_csv['text'].astype(str).values
train_texts = train_csv['text'].astype(str).values, 
train_labels = train_csv['class'].apply(lambda x: int(x == 'true')).values

### XLM-Roberta

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
tokenizer_roberta = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

In [ ]:
train_encodings_rob = tokenizer_roberta(train_texts[0].tolist(), truncation=True, padding=True)
test_encodings_rob = tokenizer_roberta(test_texts.tolist(), truncation=True, padding=True)

In [ ]:
train_dataset_rob = FNDataset(train_encodings_rob, train_labels)
test_dataset_rob = FNDataset(test_encodings_rob, [-1] * len(test_texts))

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir=global_path+'results',       
    num_train_epochs=4,              
    per_device_train_batch_size=3,  
    fp16=True,
    max_grad_norm=0.25,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=64,  
    warmup_steps=1000,              
    weight_decay=0.01,              
    logging_dir=global_path+'/logs',           
    logging_steps=25,
    learning_rate=3e-05,
    save_steps=5000,
)

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-large")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [ ]:
trainer = Trainer(
    model=model,                        
    args=training_args,                  
    train_dataset=train_dataset_rob,         
    #eval_dataset=val_dataset             
)

trainer.train()

In [ ]:
model.save_pretrained(global_path+'xlm-roberta-large5')

### Prediction

In [ ]:
test_iter_rob = torch.utils.data.DataLoader(test_dataset_rob, batch_size=32)

In [ ]:
def get_pred_distr(iter, model):
    preds = []
    for batch in tqdm(iter):
        with torch.no_grad():
            preds.append(model(
                batch['input_ids'].cuda(), 
                batch['attention_mask'].cuda()))
    return preds

preds1 = get_pred_distr(test_iter_rob, model)

In [ ]:
final_preds_rob = []
for batch in preds1:
    final_preds_rob += (torch.softmax(batch[0], dim=1) >= 0.5).cpu().numpy().astype(int).argmax(axis=1).tolist()

---

In [ ]:
test_csv['class'] = pd.Series(final_preds_rob).map({1: 'true', 0: 'fake'})
test_csv

,text,class
0,Британское министерство информации приняло дек...,fake
1,"Альбом ""Chief"" кантри-исполнителя Эрика Черча ...",true
2,Новый вид микрокредитных организаций появился ...,fake
3,Один из крупнейших торговых комплексов Москвы ...,fake
4,Министерство обороны Белоруссии приостановила ...,true
...,...,...
6019,Представители интернет-сообщества попросили Го...,true
6020,Московские кришнаиты как и было запланировано ...,true
6021,Компания Fox Broadcasting договорились с опера...,true
6022,Гражданка Великобритании Джорджина Чайлдс задо...,true


In [ ]:
test_csv.to_csv(global_path+'test.csv', index=False, sep='\t')

In [ ]:
from zipfile import ZipFile
with ZipFile(global_path+'subm.zip', 'w') as myzip:
    myzip.write(global_path+'test.csv', 'test.csv')